Cose da fare e/o controllare : 

*   Test
*   vedere se ./255 serve e per cosa si usa : si usa il rescaling oppure il batchnormalization a quanto pare



# **Librerie**

[Dataset](https://www.kaggle.com/datasets/paultimothymooney/kermany2018)

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Dropout
from sklearn.metrics import accuracy_score
np.random.seed(42)
from google.colab import drive

from matplotlib import style
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Activation, add, Add, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras import optimizers

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input


# **Train & Val Load**

In [2]:
# setup directory used in the project for training and test phase
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/dataset/OCT/'
train_dir = '/content/drive/MyDrive/dataset/OCT/train/'
test_dir = '/content/drive/MyDrive/dataset/OCT/test'
val_dir = '/content/drive/MyDrive/dataset/OCT/val/'

Mounted at /content/drive


In [13]:
train = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='grayscale',
    batch_size=64,
    image_size=(50, 50),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)




Found 83493 files belonging to 4 classes.


In [14]:

val = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='grayscale',
    batch_size=64,
    image_size=(50, 50),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 32 files belonging to 4 classes.


### **Class weight**

Distribution of the class inside the folder

In [22]:
#compute class weight train for inbalanced dataset --> estimated time for 80k documents = 3s

import pathlib

# create array with all classes in the train
summary = pd.DataFrame()
val_classes = [item.name for item in 
                 pathlib.Path('/content/drive/MyDrive/dataset/OCT/train').glob('*')]

# count the number of files in each folder
# print(val_classes)
for d in ['train']:
    for c in val_classes:
        n = len([i for i in pathlib.Path(f'/content/drive/MyDrive/dataset/OCT/{d}/{c}').glob('*')])
        summary.loc[c, d] = n
summary.style.format("{:.0f}")




['CNV', 'DME', 'DRUSEN', 'NORMAL']


,train
CNV,37214
DME,11348
DRUSEN,8616
NORMAL,26315


Compue class_weights

In [23]:
train_directory = '/content/drive/MyDrive/dataset/OCT/train/'

# ['DME', 'CNV', 'NORMAL', '.DS_Store', 'DRUSEN']
from tqdm import tqdm
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['NORMAL']:
                label = 0
            elif folderName in ['CNV']:
                label = 1
            elif folderName in ['DME']:
                label = 2
            elif folderName in ['DRUSEN']:
                label = 3
            else:
                label = 4
            for image_filename in tqdm(os.listdir(folder + folderName)):
                y.append(label)
    y = np.asarray(y)
    return y
y_train= get_data(train_directory)


from keras.utils.np_utils import to_categorical
y_trainHot = to_categorical(y_train, num_classes = 4)

from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight = "balanced", classes = np.unique(y_train),y =  y_train)
class_weights = dict(zip(np.unique(y_train), class_weight))
class_weights


100%|██████████| 26315/26315 [00:00<00:00, 891334.90it/s]


{0: 0.7932072962188865,
 1: 0.5608977804052239,
 2: 1.839376982728234,
 3: 2.4226149025069637}

# **CNN FROM SCRATCH**

## **ALEXNET**

### **Train**

In [15]:
#Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [16]:
img_size = 50
model = keras.models.Sequential()
#1st Convolutional Layer
model.add(keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu', input_shape = [img_size, img_size,1]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Dropout(0.2))
#2nd Convolutional Layer
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Dropout(0.2))
#3rd Convolutional Layer
model.add(keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Dropout(0.2))
#4th Convolutional Layer
model.add(keras.layers.Conv2D(filters=256,kernel_size=3,activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Dropout(0.2))
#Passing it to a Fully Connected layer
model.add(keras.layers.Flatten())

# 1st Fully Connected Layer
model.add(keras.layers.Dense(512,activation='relu'))

# 2st Fully Connected Layer
model.add(keras.layers.Dense(512,activation='relu'))

#Softmax layer for output
model.add(keras.layers.Dense(4,activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_4 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 24, 24, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 batch_normalization_5 (Batc  (None, 22, 22, 64)      

Training **Senza** Augmentation

In [ ]:

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = model.fit(train, epochs=12, validation_data=val , verbose = 1 , callbacks=[early_stopping] , class_weight = class_weights)


Epoch 1/12
   3/1305 [..............................] - ETA: 17:35 - loss: 2.7644 - accuracy: 0.3281

Training **Con** Augmentation

In [ ]:
#Augmentation --> farlo con il layer
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

img_height = 50
batch_size=32
epochs = 20
model.fit(aug.flow_from_directory(directory=train_dir,target_size=(img_height, img_height),batch_size=batch_size ,shuffle=True,color_mode='grayscale',seed=None,class_mode='categorical'),
          epochs=epochs,
          validation_data=val,
          class_weight = class_weights)

### **Test**

In [ ]:
model.save('/content/drive/MyDrive/model/AlexNet.h5')   # always save your weights after training or during training
#evaluation with history
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='grayscale',
    batch_size=32,
    image_size=(50, 50),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

## **XCEPTION**

# **CNN PRETRAINED**

## **EFFICENT NET**

### **Train**

In [ ]:
# epochs = 20  # @param {type: "slider", min:10, max:100}
#Augmentation --> farlo con il layer

m_channels = 3
train = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb', #--> attenzione a questo valore
    batch_size=64,
    image_size=(50, 50),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

val = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=64,
    image_size=(50, 50),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

img_dim = 50
batch_size=32
train_augmented = aug.flow_from_directory(
    directory=train_dir,
    target_size=(img_dim, img_dim),
    batch_size=batch_size ,
    shuffle=True,
    color_mode='rgb',
    seed=None,
    class_mode='categorical')

Found 83493 files belonging to 4 classes.
Found 32 files belonging to 4 classes.
Found 83493 images belonging to 4 classes.


In [ ]:
# Create Model
model_ENB0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(img_dim,img_dim,m_channels))
model_ENB0.trainable = False
model_ENB0.summary()

16705208/16705208 [==============================] - 0s 0us/step
Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50, 50, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 50, 50, 3)    0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 50, 50, 3)    7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 50, 50, 3)    0           ['normalization[0][0]']          
                    

In [ ]:
CLASSES = 4
model = Sequential()
model.add(model_ENB0)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(CLASSES,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 2, 2, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 4,054,695
Trainable params: 5,124
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
#Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)
# Pre Training
model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.Adam(learning_rate=1e-2),
              metrics=['accuracy'])

# with augmentation 
# history = model.fit(train_augmented, epochs=5 , validation_data=val , verbose = 1 , callbacks=[early_stopping] , class_weight = class_weights)

# without augmentation 
history = model.fit(train, epochs=15 , validation_data=val , verbose = 1 , callbacks=[early_stopping] , class_weight = class_weights)


In [ ]:
#evaluation with history
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
# Freezing all layers until the fifth from the last
model_ENB0.trainable = True
for layer in model_ENB0.layers[:-5]:
 layer.trainable = False

 
model_ENB0.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50, 50, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 50, 50, 3)    0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 50, 50, 3)    7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 50, 50, 3)    0           ['normalization[0][0]']          
                                                                                     

In [ ]:
#Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

# Pre Training
model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.Adam(learning_rate=1e-2),
              metrics=['accuracy'])
 

# history = model.fit(train_augmented, epochs=5 , validation_data=val , verbose = 1 , callbacks=[early_stopping] , class_weight = class_weights)
history = model.fit(train, epochs=15 , validation_data=val , verbose = 1 , callbacks=[early_stopping] , class_weight = class_weights)

Epoch 1/15
  20/1305 [..............................] - ETA: 2:33:22 - loss: 1.6823 - accuracy: 0.5000

KeyboardInterrupt: ignored

In [ ]:
#evaluation with history
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

### **Test**

# **STOP - EXECUTE ALWAYS**

In [ ]:
from google.colab import runtime
runtime.unassign()